In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [3]:
df_pivot_2 = pd.read_csv("df_pivot_2.csv")

In [4]:
df_pivot_2

,client_id,Variation,clnt_tenure_yr,age_group,confirm,start,step_1,step_2,step_3,drop_start_step_1,...,confirm_filled,time_confirm,time_start,time_step_1,time_step_2,time_step_3,error_start_step_1,error_step_1_step_2,error_step_2_step_3,error_step_3_confirm
0,555,Test,3.0,25-34,5.0,5.0,5.0,5.0,5.0,0.0,...,5.0,158.0,158.0,158.0,158.0,158.0,0,0,0,0
1,647,Test,12.0,55-64,5.0,5.0,5.0,5.0,5.0,0.0,...,5.0,377.0,377.0,377.0,377.0,377.0,0,0,0,0
2,934,Test,9.0,45-54,NaN,4.0,NaN,NaN,NaN,NaN,...,0.0,NaN,568.0,NaN,NaN,NaN,0,0,0,0
3,1028,Control,12.0,35-44,NaN,9.0,9.0,9.0,9.0,0.0,...,0.0,NaN,538.0,2690.0,1076.0,538.0,0,0,0,0
4,1104,Control,5.0,45-54,NaN,1.0,NaN,NaN,NaN,NaN,...,0.0,NaN,0.0,NaN,NaN,NaN,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
50482,9999150,Test,5.0,25-34,NaN,3.0,3.0,NaN,NaN,0.0,...,0.0,NaN,36.0,18.0,NaN,NaN,0,0,0,0
50483,9999400,Test,7.0,25-34,5.0,5.0,5.0,5.0,5.0,0.0,...,5.0,119.0,119.0,119.0,119.0,119.0,0,0,0,0
50484,9999626,Test,9.0,25-34,NaN,2.0,2.0,NaN,NaN,0.0,...,0.0,NaN,8.0,8.0,NaN,NaN,0,0,0,0
50485,9999729,Test,10.0,25-34,5.0,2.0,2.0,5.0,5.0,0.0,...,5.0,75.0,990.0,540.0,525.0,75.0,0,0,0,0


**1.- Completion Rate**

In [5]:
#Step 1: Completion Rate Hypothesis Test
###Objective: Test whether the new design (Test) has a significantly higher completion rate than the old design (Control).

# H0 : p-test = p-control --> There is no difference in completion rates between Test and Control.
# H1 : p-test > p-control --> Test has a higher completion rate than Control.

alpha = 0.05

from statsmodels.stats.proportion import proportions_ztest

# Count of completions
successes = df_pivot_2.groupby('Variation')['confirm_filled'].apply(lambda x: (x>0).sum())
# Total users
nobs = df_pivot_2.groupby('Variation')['confirm_filled'].count()

z_stat, p_value = proportions_ztest(count=successes, nobs=nobs, alternative='larger')
print(f"Z-test statistic: {z_stat:.4f}, p-value: {p_value:.4f}")


Z-test statistic: -8.8927, p-value: 1.0000


In [6]:
##Desicion making
if p_value < alpha:
    print("We reject the null hypothesis, Test completion rate is significantly higher")
else:
    print("We fail to reject the null hypothesis")

We fail to reject the null hypothesis


In [7]:
#Step 2: Completion Rate Cost-Effectiveness Threshold
###Objective: Check whether the increase in completion rate exceeds 5%, which is required to justify the redesign costs.

completion_rate = df_pivot_2.groupby('Variation')['confirm_filled'].apply(lambda x: (x>0).mean())
increase = completion_rate['Test'] - completion_rate['Control']
print(f"Completion Rate Increase: {increase*100:.2f}%")


Completion Rate Increase: 3.71%


In [8]:
if increase >= 5:
    print("✅ The redesign meets the 5% cost-effectiveness threshold.")
else:
    print("⚠️ The redesign does NOT justify the costs, as the increase is below 5%.")

⚠️ The redesign does NOT justify the costs, as the increase is below 5%.


- **Hypothesis Test (Step 1)**:
    
    - **H0**: No difference between Test and Control.
    - **H1**: Test has a higher completion rate than Control.
    - **Result**: Z = -8.8927, p = 1.000 → **Fail to reject H0.**
    - **Conclusion**: The observed higher completion rate in Test is **not statistically significant**.

- **Cost-Effectiveness Threshold (Step 2)**:
    - **Increase**: 3.71%
    - **Threshold**: 5%
    - **Conclusion: ⚠️ Redesign does NOT justify the costs**, as the increase is below 5%.

**2.- Average Age**

In [9]:
from scipy.stats import ttest_ind

#Step 3: Hypothesis Testing for Average Age
# H0: mean age is the same for Control and Test
# H1: mean age is different
df_pivot_2['age_num'] = df_pivot_2['age_group'].replace({'<25':22, '25-34':30, '35-44':40, '45-54':50, '55-64':60, '65+':70})
control_age = df_pivot_2[df_pivot_2['Variation']=='Control']['age_num'].dropna()
test_age = df_pivot_2[df_pivot_2['Variation']=='Test']['age_num'].dropna()
t_stat_age, p_value_age = ttest_ind(test_age, control_age, equal_var=False)

print("\n=== Average Age Hypothesis Test ===")
print(f"T-statistic: {t_stat_age:.4f}, p-value: {p_value_age:.4f}")
if p_value_age < 0.05:
    print("✅ Significant difference in average age between Control and Test.")
else:
    print("⚠️ No significant difference in average age.")


=== Average Age Hypothesis Test ===
T-statistic: -2.5909, p-value: 0.0096
✅ Significant difference in average age between Control and Test.


/var/folders/bp/q0jpdzq126j54kx_lzjc4dcm0000gn/T/ipykernel_44422/2647169424.py:6: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_pivot_2['age_num'] = df_pivot_2['age_group'].replace({'<25':22, '25-34':30, '35-44':40, '45-54':50, '55-64':60, '65+':70})


In [10]:
import pandas as pd
from scipy.stats import chi2_contingency

# Contingency table Between Variation and Age group
age_table = pd.crosstab(df_pivot_2['Variation'], df_pivot_2['age_group'])

print("=== Age Distribution Table ===")
print(age_table)

# Test chi-square
chi2, p_val, dof, expected = chi2_contingency(age_table)

print("\n=== Age Group Chi-Square Test ===")
print(f"Chi2 statistic: {chi2:.4f}, p-value: {p_val:.4f}, dof: {dof}")
if p_val < 0.05:
    print("✅ Significant difference in age distribution between Control and Test.")
else:
    print("⚠️ No significant difference in age distribution between Control and Test.")

print("\n=== Expected Frequencies ===")
print(pd.DataFrame(expected, index=age_table.index, columns=age_table.columns))


=== Age Distribution Table ===
age_group  25-34  35-44  45-54  55-64   65+   <25
Variation                                        
Control     4817   3966   4607   5069  3313  1754
Test        5712   4560   5353   5621  3670  2045

=== Age Group Chi-Square Test ===
Chi2 statistic: 8.7141, p-value: 0.1210, dof: 5
⚠️ No significant difference in age distribution between Control and Test.

=== Expected Frequencies ===
age_group        25-34       35-44        45-54        55-64          65+  \
Variation                                                                   
Control    4906.317547  3972.95692  4641.174164  4981.340543  3253.947709   
Test       5622.682453  4553.04308  5318.825836  5708.659457  3729.052291   

age_group          <25  
Variation               
Control    1770.263117  
Test       2028.736883  


- **T-Test:** t = -2.5909, p = 0.0096 → ✅ Significant difference in average age between Test and Control.

- **Chi-Square Test on Age Distribution**: Chi2 = 8.7141, p = 0.121 → ⚠️ No significant difference in age distribution between groups.

- **Conclusion**: Although the mean age differs slightly, the overall age distribution is similar between Test and Control.

**3.- Tenure Distribution**

In [11]:
### 2. Distribution for tenure (Chi-square)
tenure_table = pd.crosstab(df_pivot_2['Variation'], df_pivot_2['tenure_group'])
chi2, p_val, dof, expected = chi2_contingency(tenure_table)

print("\n=== Tenure Distribution Chi-Square Test ===")
print(tenure_table)
print(f"Chi2: {chi2:.4f}, p-value: {p_val:.4f}")
print("✅ Significant" if p_val < 0.05 else "⚠️ Not significant")


=== Tenure Distribution Chi-Square Test ===
tenure_group  less-tenured  more-tenured
Variation                               
Control              11813         11713
Test                 13349         13612
Chi2: 2.4364, p-value: 0.1185
⚠️ Not significant


- **Chi-Square Test**: Chi2 = 2.4364, p = 0.1185 → ⚠️ Not significant.
- **Conclusion**: The split between less-tenured and more-tenured users is similar across Test and Control.

In [12]:
df_abtest = pd.read_csv("df_abtest.csv")

In [13]:
df_abtest.columns

Index(['client_id', 'Variation', 'clnt_tenure_yr', 'clnt_tenure_mnth',
       'clnt_age', 'gendr', 'num_accts_x', 'bal_x', 'calls_6_mnth_x',
       'logons_6_mnth_x', 'visitor_id', 'visit_id', 'process_step',
       'date_time', 'source', 'num_steps', 'session_duration_sec',
       'num_accts_y', 'bal_y', 'calls_6_mnth_y', 'logons_6_mnth_y',
       'num_sessions'],
      dtype='object')

In [14]:
df_pivot_2.columns

Index(['client_id', 'Variation', 'clnt_tenure_yr', 'age_group', 'confirm',
       'start', 'step_1', 'step_2', 'step_3', 'drop_start_step_1',
       'drop_step_1_step_2', 'drop_step_2_step_3', 'drop_step_3_confirm',
       'tenure_group', 'confirm_filled', 'time_confirm', 'time_start',
       'time_step_1', 'time_step_2', 'time_step_3', 'error_start_step_1',
       'error_step_1_step_2', 'error_step_2_step_3', 'error_step_3_confirm',
       'age_num'],
      dtype='object')

In [15]:
# Merge both datasets on client_id and Variation
df_merged = df_pivot_2.merge(
    df_abtest[['client_id','Variation','session_duration_sec','num_sessions']], 
    on=['client_id','Variation'],
    how='inner'
)

print("✅ df_merged ready with", df_merged.shape[0], "rows")
print(df_merged.head())

✅ df_merged ready with 317123 rows
   client_id Variation  clnt_tenure_yr age_group  confirm  start  step_1  \
0        555      Test             3.0     25-34      5.0    5.0     5.0   
1        555      Test             3.0     25-34      5.0    5.0     5.0   
2        555      Test             3.0     25-34      5.0    5.0     5.0   
3        555      Test             3.0     25-34      5.0    5.0     5.0   
4        555      Test             3.0     25-34      5.0    5.0     5.0   

   step_2  step_3  drop_start_step_1  ...  time_step_1  time_step_2  \
0     5.0     5.0                0.0  ...        158.0        158.0   
1     5.0     5.0                0.0  ...        158.0        158.0   
2     5.0     5.0                0.0  ...        158.0        158.0   
3     5.0     5.0                0.0  ...        158.0        158.0   
4     5.0     5.0                0.0  ...        158.0        158.0   

   time_step_3 error_start_step_1  error_step_1_step_2  error_step_2_step_3  \
0 

**4.- Session Duration**

In [16]:
from scipy import stats

# --- Session Duration Test ---
print("\n=== Session Duration Hypothesis Test ===")

control_dur = df_merged[df_merged['Variation'] == 'Control']['session_duration_sec'].dropna()
test_dur = df_merged[df_merged['Variation'] == 'Test']['session_duration_sec'].dropna()

t_stat, p_val = stats.ttest_ind(control_dur, test_dur, equal_var=False)  # Welch’s t-test
print(f"T-statistic: {t_stat:.4f}, p-value: {p_val:.4f}")
if p_val < 0.05:
    print("✅ Significant difference in session duration between Control and Test.")
else:
    print("❌ No significant difference in session duration between Control and Test.")



=== Session Duration Hypothesis Test ===
T-statistic: -20.0454, p-value: 0.0000
✅ Significant difference in session duration between Control and Test.


In [17]:
# === SESSION DURATION TEST BY AGE GROUP ===
print("=== Session Duration Test by Age Group ===")
for age in df_merged['age_group'].unique():
    control_dur = df_merged[(df_merged['Variation']=='Control') & (df_merged['age_group']==age)]['session_duration_sec'].dropna()
    test_dur = df_merged[(df_merged['Variation']=='Test') & (df_merged['age_group']==age)]['session_duration_sec'].dropna()
    t_stat, p_val = stats.ttest_ind(control_dur, test_dur, equal_var=False)
    print(f"Age Group {age}: t={t_stat:.4f}, p={p_val:.4f}")

=== Session Duration Test by Age Group ===
Age Group 25-34: t=-5.3870, p=0.0000
Age Group 55-64: t=-15.4652, p=0.0000
Age Group 45-54: t=-2.6498, p=0.0081
Age Group 35-44: t=-3.9571, p=0.0001
Age Group <25: t=-10.8114, p=0.0000
Age Group 65+: t=-12.4318, p=0.0000


In [18]:
# === SESSION DURATION TEST BY TENURE GROUP ===
print("\n=== Session Duration Test by Tenure Group ===")
for tenure in df_merged['tenure_group'].unique():
    control_dur = df_merged[(df_merged['Variation']=='Control') & (df_merged['tenure_group']==tenure)]['session_duration_sec'].dropna()
    test_dur = df_merged[(df_merged['Variation']=='Test') & (df_merged['tenure_group']==tenure)]['session_duration_sec'].dropna()
    t_stat, p_val = stats.ttest_ind(control_dur, test_dur, equal_var=False)
    print(f"Tenure Group {tenure}: t={t_stat:.4f}, p={p_val:.4f}")



=== Session Duration Test by Tenure Group ===
Tenure Group less-tenured: t=-11.3996, p=0.0000
Tenure Group more-tenured: t=-17.5140, p=0.0000


- **Overall T-Test**: t = -20.0454, p < 0.001 → ✅ Significant difference; users spend more time in Test.
- **By Age Group**: Significant increases in all age groups, especially <25, 55-64, and 65+.
- **By Tenure Group**: Significant increases in both less-tenured and more-tenured groups.
- **Conclusion**: Test design **increases session duration**, particularly for younger and older users, and for long-tenured clients.

**5.- Error Rate**

In [19]:
# --- Error Rate Test ---
print("\n=== Error Rate Hypothesis Test ===")

# Define error-related columns from df_pivot_2
error_cols = [
    'error_start_step_1',
    'error_step_1_step_2',
    'error_step_2_step_3',
    'error_step_3_confirm'
]

# Calculate total errors per client
df_merged['total_errors'] = df_merged[error_cols].sum(axis=1)

control_err = df_merged[df_merged['Variation'] == 'Control']['total_errors']
test_err = df_merged[df_merged['Variation'] == 'Test']['total_errors']

t_stat_err, p_val_err = stats.ttest_ind(control_err, test_err, equal_var=False)
print(f"T-statistic: {t_stat_err:.4f}, p-value: {p_val_err:.4f}")
if p_val_err < 0.05:
    print("✅ Significant difference in error rate between Control and Test.")
else:
    print("❌ No significant difference in error rate between Control and Test.")


=== Error Rate Hypothesis Test ===
T-statistic: -48.4221, p-value: 0.0000
✅ Significant difference in error rate between Control and Test.


In [20]:
# === ERROR RATE TEST BY AGE GROUP ===
print("\n=== Error Rate Test by Age Group ===")
for age in df_merged['age_group'].unique():
    control_err = df_merged[(df_merged['Variation']=='Control') & (df_merged['age_group']==age)]['total_errors']
    test_err = df_merged[(df_merged['Variation']=='Test') & (df_merged['age_group']==age)]['total_errors']
    t_stat, p_val = stats.ttest_ind(control_err, test_err, equal_var=False)
    print(f"Age Group {age}: t={t_stat:.4f}, p={p_val:.4f}")


=== Error Rate Test by Age Group ===
Age Group 25-34: t=-12.1706, p=0.0000
Age Group 55-64: t=-27.6511, p=0.0000
Age Group 45-54: t=-20.9647, p=0.0000
Age Group 35-44: t=-19.4594, p=0.0000
Age Group <25: t=-6.4526, p=0.0000
Age Group 65+: t=-24.8998, p=0.0000


In [21]:
# === ERROR RATE TEST BY TENURE GROUP ===
print("\n=== Error Rate Test by Tenure Group ===")
for tenure in df_merged['tenure_group'].unique():
    control_err = df_merged[(df_merged['Variation']=='Control') & (df_merged['tenure_group']==tenure)]['total_errors']
    test_err = df_merged[(df_merged['Variation']=='Test') & (df_merged['tenure_group']==tenure)]['total_errors']
    t_stat, p_val = stats.ttest_ind(control_err, test_err, equal_var=False)
    print(f"Tenure Group {tenure}: t={t_stat:.4f}, p={p_val:.4f}")


=== Error Rate Test by Tenure Group ===
Tenure Group less-tenured: t=-36.2220, p=0.0000
Tenure Group more-tenured: t=-32.2341, p=0.0000


- **Overall T-Test**: t = -48.4221, p < 0.001 → ✅ Significant difference; error rate is higher in Test.
- **By Age Group**: All age groups show significantly higher errors in Test, especially 55-64 and 65+.
- **By Tenure Group**: Both less-tenured and more-tenured users have significantly higher errors in Test.
- **Conclusion**: Test design **increases errors**, particularly at the final confirm step and among older or more experienced users.

**✅ Final Summary**

- **Completion Rate**: Slight improvement in Test, **not statistically significant**, below 5% cost-effectiveness threshold.

- **Session Duration**: Test increases time spent per step, especially for <25, 55+, 65+, and long-tenured users.

- **Error Rates**: Test shows higher errors across all steps, **particularly the confirm step**.

- **Overall**: Test design slightly improves completion but reduces efficiency and increases errors in key user segments → careful consideration needed before adoption.

In [22]:
# ------------------------------
# 1. Prepare summary for Variation
# ------------------------------
variation_summary = pd.DataFrame({
    "Completion Rate": completion_rate,   # from your previous calculations
    "Avg Duration (sec)": df_pivot_2[['time_start','time_step_1','time_step_2','time_step_3','time_confirm']].mean(axis=1).groupby(df_pivot_2['Variation']).mean(),
    "Avg Error Rate": df_pivot_2[['error_start_step_1','error_step_1_step_2','error_step_2_step_3','error_step_3_confirm']].mean(axis=1).groupby(df_pivot_2['Variation']).mean()
})

# Round for readability
variation_summary = variation_summary.round(3)
print("=== Summary by Variation ===")
print(variation_summary)

# ------------------------------
# 2. Prepare summary by Age Group (fixed)
# ------------------------------
age_summary = []

for var in df_pivot_2['Variation'].unique():
    for age in df_pivot_2['age_group'].unique():
        subset = df_pivot_2[(df_pivot_2['Variation']==var) & (df_pivot_2['age_group']==age)]
        completion = (subset['confirm_filled']>0).mean()
        avg_duration = subset[['time_start','time_step_1','time_step_2','time_step_3','time_confirm']].mean(axis=1).mean()
        avg_error = subset[['error_start_step_1','error_step_1_step_2','error_step_2_step_3','error_step_3_confirm']].mean(axis=1).mean()
        # Append as a dict
        age_summary.append({
            'Variation': var,
            'Age Group': age,
            'Completion Rate': round(completion, 3),
            'Avg Duration (sec)': round(avg_duration, 3),
            'Avg Error Rate': round(avg_error, 3)
        })

age_summary = pd.DataFrame(age_summary)
age_summary.set_index(['Variation','Age Group'], inplace=True)
print(age_summary)

# ------------------------------
# 3. Prepare summary by Tenure Group
# ------------------------------
tenure_summary = []

for var in df_pivot_2['Variation'].unique():
    for tenure in df_pivot_2['tenure_group'].unique():
        subset = df_pivot_2[(df_pivot_2['Variation']==var) & (df_pivot_2['tenure_group']==tenure)]
        completion = (subset['confirm_filled']>0).mean()
        avg_duration = subset[['time_start','time_step_1','time_step_2','time_step_3','time_confirm']].mean(axis=1).mean()
        avg_error = subset[['error_start_step_1','error_step_1_step_2','error_step_2_step_3','error_step_3_confirm']].mean(axis=1).mean()
        tenure_summary.append({
            'Variation': var,
            'Tenure Group': tenure,
            'Completion Rate': round(completion, 3),
            'Avg Duration (sec)': round(avg_duration, 3),
            'Avg Error Rate': round(avg_error, 3)
        })

tenure_summary = pd.DataFrame(tenure_summary)
tenure_summary.set_index(['Variation','Tenure Group'], inplace=True)
print(tenure_summary)


=== Summary by Variation ===
           Completion Rate  Avg Duration (sec)  Avg Error Rate
Variation                                                     
Control              0.656             604.918           0.007
Test                 0.693             773.191           0.015
                     Completion Rate  Avg Duration (sec)  Avg Error Rate
Variation Age Group                                                     
Test      25-34                0.716             515.109           0.011
          55-64                0.696             994.762           0.019
          45-54                0.686             832.251           0.017
          35-44                0.704             582.669           0.013
          <25                  0.709             582.254           0.011
          65+                  0.639            1092.483           0.019
Control   25-34                0.670             449.343           0.006
          55-64                0.661             714.427      

In [23]:
import pandas as pd

# ------------------------------
# 1️⃣ Summary by Variation
# ------------------------------
variation_summary = pd.DataFrame({
    "Variation": df_pivot_2['Variation'].unique()
})

# Completion Rate
completion_rate = df_pivot_2.groupby('Variation')['confirm_filled'].apply(lambda x: (x>0).mean()).reset_index(drop=True)
variation_summary['Completion Rate'] = (completion_rate*100).round(2)  # as percentage

# Avg Duration (sec)
avg_duration = df_pivot_2[['Variation','time_start','time_step_1','time_step_2','time_step_3','time_confirm']].copy()
avg_duration['Avg Duration (sec)'] = avg_duration[['time_start','time_step_1','time_step_2','time_step_3','time_confirm']].mean(axis=1)
avg_duration_summary = avg_duration.groupby('Variation')['Avg Duration (sec)'].mean().reset_index(drop=True)
variation_summary['Avg Duration (sec)'] = avg_duration_summary.round(2)

# Avg Error Rate
avg_error = df_pivot_2[['Variation','error_start_step_1','error_step_1_step_2','error_step_2_step_3','error_step_3_confirm']].copy()
avg_error['Avg Error Rate'] = avg_error[['error_start_step_1','error_step_1_step_2','error_step_2_step_3','error_step_3_confirm']].mean(axis=1)
avg_error_summary = avg_error.groupby('Variation')['Avg Error Rate'].mean().reset_index(drop=True)
variation_summary['Avg Error Rate'] = avg_error_summary.round(3)

# Export CSV
variation_summary.to_csv("tableau_summary_by_variation.csv", index=False)
print("✅ CSV for Variation saved!")

# ------------------------------
# 2️⃣ Summary by Age Group
# ------------------------------
age_summary = []

for var in df_pivot_2['Variation'].unique():
    for age in df_pivot_2['age_group'].unique():
        subset = df_pivot_2[(df_pivot_2['Variation']==var) & (df_pivot_2['age_group']==age)]
        completion = (subset['confirm_filled']>0).mean()
        avg_duration = subset[['time_start','time_step_1','time_step_2','time_step_3','time_confirm']].mean(axis=1).mean()
        avg_error = subset[['error_start_step_1','error_step_1_step_2','error_step_2_step_3','error_step_3_confirm']].mean(axis=1).mean()
        age_summary.append({
            'Variation': var,
            'Age Group': age,
            'Completion Rate': round(completion*100,2),
            'Avg Duration (sec)': round(avg_duration,2),
            'Avg Error Rate': round(avg_error,3)
        })

age_summary = pd.DataFrame(age_summary)
age_summary.to_csv("tableau_summary_by_age.csv", index=False)
print("✅ CSV for Age Group saved!")

# ------------------------------
# 3️⃣ Summary by Tenure Group
# ------------------------------
tenure_summary = []

for var in df_pivot_2['Variation'].unique():
    for tenure in df_pivot_2['tenure_group'].unique():
        subset = df_pivot_2[(df_pivot_2['Variation']==var) & (df_pivot_2['tenure_group']==tenure)]
        completion = (subset['confirm_filled']>0).mean()
        avg_duration = subset[['time_start','time_step_1','time_step_2','time_step_3','time_confirm']].mean(axis=1).mean()
        avg_error = subset[['error_start_step_1','error_step_1_step_2','error_step_2_step_3','error_step_3_confirm']].mean(axis=1).mean()
        tenure_summary.append({
            'Variation': var,
            'Tenure Group': tenure,
            'Completion Rate': round(completion*100,2),
            'Avg Duration (sec)': round(avg_duration,2),
            'Avg Error Rate': round(avg_error,3)
        })

tenure_summary = pd.DataFrame(tenure_summary)
tenure_summary.to_csv("tableau_summary_by_tenure.csv", index=False)
print("✅ CSV for Tenure Group saved!")

# ------------------------------
# 4️⃣ Optional: Long Format Client-Level Data (for advanced Tableau analysis)
# ------------------------------
df_pivot_2.to_csv("tableau_client_long_format.csv", index=False)
print("✅ Client-level long format CSV saved!")


✅ CSV for Variation saved!
✅ CSV for Age Group saved!
✅ CSV for Tenure Group saved!
✅ Client-level long format CSV saved!


In [24]:
import pandas as pd

# Columns that indicate reaching a step (any non-NaN or non-zero)
steps = ['start', 'step_1', 'step_2', 'step_3', 'confirm']

# Create a long format dataframe for steps reached
records = []
for _, row in df_pivot_2.iterrows():
    for step in steps:
        reached = 1 if pd.notna(row[step]) and row[step] > 0 else 0
        records.append({
            'client_id': row['client_id'],
            'Variation': row['Variation'],
            'Age Group': row['age_group'],
            'Tenure Group': row['tenure_group'],
            'process_step': step,
            'reached_step': reached
        })

df_steps_long = pd.DataFrame(records)

# Save to CSV to import into Tableau
df_steps_long.to_csv("tableau_steps_long.csv", index=False)
print(df_steps_long.head())


   client_id Variation Age Group  Tenure Group process_step  reached_step
0        555      Test     25-34  less-tenured        start             1
1        555      Test     25-34  less-tenured       step_1             1
2        555      Test     25-34  less-tenured       step_2             1
3        555      Test     25-34  less-tenured       step_3             1
4        555      Test     25-34  less-tenured      confirm             1


In [25]:
import pandas as pd

# Crear columna auxiliar con el error medio por fila
df_pivot_2["row_error_mean"] = df_pivot_2[
    ["error_start_step_1","error_step_1_step_2","error_step_2_step_3","error_step_3_confirm"]
].mean(axis=1)

# Crear columna auxiliar con la duración media por fila
df_pivot_2["row_duration_mean"] = df_pivot_2[
    ["time_start","time_step_1","time_step_2","time_step_3","time_confirm"]
].mean(axis=1)


# ------------------------------
# 1. Summary by Variation
# ------------------------------
variation_summary = (
    df_pivot_2.groupby("Variation")
    .agg(
        Completion_Rate=("confirm_filled", lambda x: (x > 0).mean()),
        Avg_Duration_sec=("row_duration_mean", "mean"),
        Avg_Error_Rate=("row_error_mean", "mean")
    )
    .reset_index()
)

variation_summary = variation_summary.round(3)
variation_summary.to_csv("variation_summary.csv", index=False)


# ------------------------------
# 2. Summary by Age Group
# ------------------------------
age_summary = (
    df_pivot_2.groupby(["Variation","age_group"])
    .agg(
        Completion_Rate=("confirm_filled", lambda x: (x > 0).mean()),
        Avg_Duration_sec=("row_duration_mean", "mean"),
        Avg_Error_Rate=("row_error_mean", "mean")
    )
    .reset_index()
)

age_summary = age_summary.round(3)
age_summary.to_csv("age_summary.csv", index=False)


# ------------------------------
# 3. Summary by Tenure Group
# ------------------------------
tenure_summary = (
    df_pivot_2.groupby(["Variation","tenure_group"])
    .agg(
        Completion_Rate=("confirm_filled", lambda x: (x > 0).mean()),
        Avg_Duration_sec=("row_duration_mean", "mean"),
        Avg_Error_Rate=("row_error_mean", "mean")
    )
    .reset_index()
)

tenure_summary = tenure_summary.round(3)
tenure_summary.to_csv("tenure_summary.csv", index=False)


print("✅ CSV exportados: variation_summary.csv, age_summary.csv, tenure_summary.csv")


✅ CSV exportados: variation_summary.csv, age_summary.csv, tenure_summary.csv


In [26]:
import pandas as pd

# --- Step 1: Select relevant columns ---
# session durations per step, error rates per step, plus client info
cols_duration = ['client_id', 'Variation', 'age_group', 'tenure_group',
                 'time_start', 'time_step_1', 'time_step_2', 'time_step_3', 'time_confirm']

cols_error = ['error_start_step_1', 'error_step_1_step_2', 'error_step_2_step_3', 'error_step_3_confirm']

df_tableau = df_merged[cols_duration + cols_error].copy()

# --- Step 2: Melt session duration columns into long format ---
df_duration_long = df_tableau.melt(
    id_vars=['client_id', 'Variation', 'age_group', 'tenure_group'],
    value_vars=['time_start', 'time_step_1', 'time_step_2', 'time_step_3', 'time_confirm'],
    var_name='process_step',
    value_name='session_duration_sec'
)

# Optional: clean step names
df_duration_long['process_step'] = df_duration_long['process_step'].str.replace('time_', '')

# --- Step 3: Melt error columns into long format ---
df_error_long = df_tableau.melt(
    id_vars=['client_id', 'Variation', 'age_group', 'tenure_group'],
    value_vars=['error_start_step_1', 'error_step_1_step_2', 'error_step_2_step_3', 'error_step_3_confirm'],
    var_name='process_step',
    value_name='error'
)

# Optional: clean step names to match session duration
df_error_long['process_step'] = df_error_long['process_step'].str.replace('error_', '').str.replace('_', '')

# --- Step 4: Merge session duration and error long tables ---
df_long = pd.merge(
    df_duration_long,
    df_error_long,
    on=['client_id', 'Variation', 'age_group', 'tenure_group', 'process_step'],
    how='left'
)

# --- Step 5: Add completion flag ---
# If 'process_step' is 'confirm' and session_duration_sec > 0, then completed
df_long['completed'] = df_long.apply(
    lambda row: 1 if row['process_step']=='confirm' and row['session_duration_sec']>0 else 0,
    axis=1
)

# --- Step 6: Export to CSV for Tableau ---
df_long.to_csv('tableau_client_long_format.csv', index=False)

print("Long-format table ready for Tableau!")
print(df_long.head())


Long-format table ready for Tableau!
   client_id Variation age_group  tenure_group process_step  \
0        555      Test     25-34  less-tenured        start   
1        555      Test     25-34  less-tenured        start   
2        555      Test     25-34  less-tenured        start   
3        555      Test     25-34  less-tenured        start   
4        555      Test     25-34  less-tenured        start   

   session_duration_sec  error  completed  
0                 158.0    NaN          0  
1                 158.0    NaN          0  
2                 158.0    NaN          0  
3                 158.0    NaN          0  
4                 158.0    NaN          0  


In [27]:
import pandas as pd

# --- Step 1: Define the main df ---
df = df_merged.copy()  # Make sure df_merged has age_group and tenure_group

# --- Step 2: Completion rate summary ---
# Fill missing confirm with 0
df['confirm_filled'] = df['time_confirm'].fillna(0)
df['completed'] = df['confirm_filled'].apply(lambda x: 1 if x>0 else 0)

# Completion rate by Variation
completion_var = df.groupby('Variation')['completed'].mean().reset_index()
completion_var.rename(columns={'completed':'completion_rate'}, inplace=True)

# Completion rate by Age Group
completion_age = df.groupby(['Variation','age_group'])['completed'].mean().reset_index()
completion_age.rename(columns={'completed':'completion_rate'}, inplace=True)

# Completion rate by Tenure Group
completion_tenure = df.groupby(['Variation','tenure_group'])['completed'].mean().reset_index()
completion_tenure.rename(columns={'completed':'completion_rate'}, inplace=True)

# --- Step 3: Session duration summary ---
session_cols = ['time_start','time_step_1','time_step_2','time_step_3','time_confirm']

# Melt session durations
df_duration_long = df.melt(
    id_vars=['client_id','Variation','age_group','tenure_group'],
    value_vars=session_cols,
    var_name='process_step',
    value_name='duration_sec'
)
df_duration_long['process_step'] = df_duration_long['process_step'].str.replace('time_','')

# Aggregate avg duration
avg_duration_var = df_duration_long.groupby(['Variation','process_step'])['duration_sec'].mean().reset_index()
avg_duration_age = df_duration_long.groupby(['Variation','age_group','process_step'])['duration_sec'].mean().reset_index()
avg_duration_tenure = df_duration_long.groupby(['Variation','tenure_group','process_step'])['duration_sec'].mean().reset_index()

# --- Step 4: Error rate summary ---
error_cols = ['error_start_step_1','error_step_1_step_2','error_step_2_step_3','error_step_3_confirm']

df_error_long = df.melt(
    id_vars=['client_id','Variation','age_group','tenure_group'],
    value_vars=error_cols,
    var_name='process_step',
    value_name='error_rate'
)
df_error_long['process_step'] = df_error_long['process_step'].str.replace('error_','').str.replace('_','')

# Aggregate avg error rate
avg_error_var = df_error_long.groupby(['Variation','process_step'])['error_rate'].mean().reset_index()
avg_error_age = df_error_long.groupby(['Variation','age_group','process_step'])['error_rate'].mean().reset_index()
avg_error_tenure = df_error_long.groupby(['Variation','tenure_group','process_step'])['error_rate'].mean().reset_index()

# --- Step 5: Export CSVs ---
completion_var.to_csv('tableau_completion_by_variation.csv', index=False)
completion_age.to_csv('tableau_completion_by_age.csv', index=False)
completion_tenure.to_csv('tableau_completion_by_tenure.csv', index=False)

avg_duration_var.to_csv('tableau_duration_by_variation.csv', index=False)
avg_duration_age.to_csv('tableau_duration_by_age.csv', index=False)
avg_duration_tenure.to_csv('tableau_duration_by_tenure.csv', index=False)

avg_error_var.to_csv('tableau_error_by_variation.csv', index=False)
avg_error_age.to_csv('tableau_error_by_age.csv', index=False)
avg_error_tenure.to_csv('tableau_error_by_tenure.csv', index=False)

print("All pre-aggregated summary tables are ready for Tableau!")


All pre-aggregated summary tables are ready for Tableau!


In [28]:
# Reproducible export pipeline -> create long step table + aggregated CSVs for Tableau
import pandas as pd
import numpy as np

# --- SETTINGS: steps order (important) ---
steps = ['start', 'step_1', 'step_2', 'step_3', 'confirm']

# --- 0. Quick checks (optional) ---
print("Rows in df_pivot_2:", len(df_pivot_2))
print("Unique clients:", df_pivot_2['client_id'].nunique())
print("Variations:", df_pivot_2['Variation'].value_counts().to_dict())

# --- 1. Create client-level long table with reached_step (1 if user reached that step) ---
long_frames = []
for step in steps:
    tmp = df_pivot_2[['client_id','Variation','age_group','tenure_group', step]].copy()
    tmp = tmp.rename(columns={step: 'step_value'})
    tmp['process_step'] = step
    # Define reached: not null AND >0 (adjust if your step_value indicates presence differently)
    # If some step_value can be 0 and still considered "reached", change condition accordingly.
    tmp['reached_step'] = np.where(tmp['step_value'].notna() & (tmp['step_value'] > 0), 1, 0)
    # Keep any duration per step if you have time_* columns named similarly; if not, leave NaN
    # If you have time columns named time_start etc, we can map them later.
    long_frames.append(tmp[['client_id','Variation','age_group','tenure_group','process_step','reached_step']])

df_steps_long = pd.concat(long_frames, ignore_index=True)

# Sanity: each client should have one row per step
n_clients = df_pivot_2['client_id'].nunique()
expected_rows = n_clients * len(steps)
print("Expected rows (clients * steps):", expected_rows, "-> long rows:", len(df_steps_long))

# Save client-level long table
df_steps_long.to_csv("tableau_steps_long_corrected.csv", index=False)
print("Saved tableau_steps_long_corrected.csv")

# --- 2. Aggregations: counts and percent reached by Variation & Step ---
# Count of reached per variation-step
counts_var_step = (
    df_steps_long.groupby(['Variation','process_step'])['reached_step']
    .agg(n_reached='sum')
    .reset_index()
)

# denominator per variation: number of unique clients in that variation
denom_var = df_pivot_2.groupby('Variation')['client_id'].nunique().reset_index().rename(columns={'client_id':'n_clients'})
counts_var_step = counts_var_step.merge(denom_var, on='Variation', how='left')
counts_var_step['pct_reached'] = counts_var_step['n_reached'] / counts_var_step['n_clients']

# Order process_step as categorical for later use (and to ensure CSV shows human order)
counts_var_step['process_step'] = pd.Categorical(counts_var_step['process_step'], categories=steps, ordered=True)
counts_var_step = counts_var_step.sort_values(['Variation','process_step'])

counts_var_step.to_csv("tableau_step_counts_by_variation.csv", index=False)
print("Saved tableau_step_counts_by_variation.csv")

# --- 3. Aggregations by Variation x Age Group x Step ---
counts_var_age_step = (
    df_steps_long.groupby(['Variation','age_group','process_step'])['reached_step']
    .agg(n_reached='sum')
    .reset_index()
)

# denom per variation-age for percentages
denom_var_age = df_pivot_2.groupby(['Variation','age_group'])['client_id'].nunique().reset_index().rename(columns={'client_id':'n_clients'})
counts_var_age_step = counts_var_age_step.merge(denom_var_age, on=['Variation','age_group'], how='left')
counts_var_age_step['pct_reached'] = counts_var_age_step['n_reached'] / counts_var_age_step['n_clients']

counts_var_age_step['process_step'] = pd.Categorical(counts_var_age_step['process_step'], categories=steps, ordered=True)
counts_var_age_step = counts_var_age_step.sort_values(['Variation','age_group','process_step'])
counts_var_age_step.to_csv("tableau_step_counts_by_age.csv", index=False)
print("Saved tableau_step_counts_by_age.csv")

# --- 4. Aggregations by Variation x Tenure Group x Step ---
counts_var_tenure_step = (
    df_steps_long.groupby(['Variation','tenure_group','process_step'])['reached_step']
    .agg(n_reached='sum')
    .reset_index()
)

denom_var_tenure = df_pivot_2.groupby(['Variation','tenure_group'])['client_id'].nunique().reset_index().rename(columns={'client_id':'n_clients'})
counts_var_tenure_step = counts_var_tenure_step.merge(denom_var_tenure, on=['Variation','tenure_group'], how='left')
counts_var_tenure_step['pct_reached'] = counts_var_tenure_step['n_reached'] / counts_var_tenure_step['n_clients']

counts_var_tenure_step['process_step'] = pd.Categorical(counts_var_tenure_step['process_step'], categories=steps, ordered=True)
counts_var_tenure_step = counts_var_tenure_step.sort_values(['Variation','tenure_group','process_step'])
counts_var_tenure_step.to_csv("tableau_step_counts_by_tenure.csv", index=False)
print("Saved tableau_step_counts_by_tenure.csv")

# --- 5. Progression/Retention between steps (Variation-level) ---
# compute pct_reached pivot-like structure quickly
pivot_var = counts_var_step.pivot(index='Variation', columns='process_step', values='n_reached').reset_index()
# safe fill zero for missing
pivot_var = pivot_var.fillna(0)

# compute retention from step i to step i+1 = n_reached(step_{i+1}) / n_reached(step_i)
# careful with zeros: where n_reached(step_i)==0 set NaN
retentions = []
for var in pivot_var['Variation']:
    row = pivot_var[pivot_var['Variation']==var].iloc[0]
    ret = {'Variation': var}
    for i in range(len(steps)-1):
        s_prev = steps[i]
        s_next = steps[i+1]
        prev_n = row.get(s_prev, 0)
        next_n = row.get(s_next, 0)
        if prev_n > 0:
            ret_rate = next_n / prev_n
        else:
            ret_rate = np.nan
        ret[f'retention_{s_prev}_to_{s_next}'] = ret_rate
    retentions.append(ret)
retention_df = pd.DataFrame(retentions)
retention_df.to_csv("tableau_step_retention_by_variation.csv", index=False)
print("Saved tableau_step_retention_by_variation.csv")

# --- 6. Sanity checks prints (small) ---
print("\nSanity checks (Variation-level % reached):")
print(counts_var_step[['Variation','process_step','n_reached','n_clients','pct_reached']].to_string(index=False))

print("\nSanity checks (by age, sample):")
# show a small sample for one variation and one age
print(counts_var_age_step[counts_var_age_step['Variation']=='Control'].head(10).to_string(index=False))

# --- 7. (Optional) include session duration per step in a long format for Tableau plotting time-by-step
# If df_pivot_2 has time_x columns (time_start etc) we melt them
time_cols = ['time_start','time_step_1','time_step_2','time_step_3','time_confirm']
if set(time_cols).issubset(df_pivot_2.columns):
    df_time_long = df_pivot_2[['client_id','Variation','age_group','tenure_group'] + time_cols].melt(
        id_vars=['client_id','Variation','age_group','tenure_group'],
        value_vars=time_cols,
        var_name='process_step',
        value_name='duration_sec'
    )
    df_time_long['process_step'] = df_time_long['process_step'].str.replace('time_','')
    df_time_long.to_csv("tableau_duration_long_by_client.csv", index=False)
    # also aggregate averages
    avg_time_by_var_step = df_time_long.groupby(['Variation','process_step'])['duration_sec'].mean().reset_index()
    avg_time_by_var_step.to_csv("tableau_avg_duration_by_variation_step.csv", index=False)
    print("Saved duration long and avg duration CSVs.")
else:
    print("time_cols not present in df_pivot_2; skipping duration exports.")

# --- 8. (Optional) include error per step long format if needed ---
error_cols = ['error_start_step_1','error_step_1_step_2','error_step_2_step_3','error_step_3_confirm']
if set(error_cols).issubset(df_pivot_2.columns):
    df_error_long = df_pivot_2[['client_id','Variation','age_group','tenure_group'] + error_cols].melt(
        id_vars=['client_id','Variation','age_group','tenure_group'],
        value_vars=error_cols,
        var_name='process_step',
        value_name='error'
    )
    # normalize names to match steps (optional)
    df_error_long['process_step'] = df_error_long['process_step'].str.replace('error_','').str.replace('_','')
    df_error_long.to_csv("tableau_error_long_by_client.csv", index=False)
    # aggregate
    avg_error_by_var_step = df_error_long.groupby(['Variation','process_step'])['error'].mean().reset_index()
    avg_error_by_var_step.to_csv("tableau_avg_error_by_variation_step.csv", index=False)
    print("Saved error long and avg error CSVs.")
else:
    print("error_cols not present in df_pivot_2; skipping error exports.")

print("\nAll CSVs for step-level analysis have been created. Import these into Tableau:")
print("- tableau_steps_long_corrected.csv")
print("- tableau_step_counts_by_variation.csv")
print("- tableau_step_counts_by_age.csv")
print("- tableau_step_counts_by_tenure.csv")
print("- tableau_step_retention_by_variation.csv")
print("- (optional) tableau_duration_long_by_client.csv, tableau_avg_duration_by_variation_step.csv")
print("- (optional) tableau_error_long_by_client.csv, tableau_avg_error_by_variation_step.csv")


Rows in df_pivot_2: 50487
Unique clients: 50487
Variations: {'Test': 26961, 'Control': 23526}
Expected rows (clients * steps): 252435 -> long rows: 252435
Saved tableau_steps_long_corrected.csv
Saved tableau_step_counts_by_variation.csv
Saved tableau_step_counts_by_age.csv
Saved tableau_step_counts_by_tenure.csv
Saved tableau_step_retention_by_variation.csv

Sanity checks (Variation-level % reached):
Variation process_step  n_reached  n_clients  pct_reached
  Control        start      23391      23526     0.994262
  Control       step_1      20146      23526     0.856329
  Control       step_2      18644      23526     0.792485
  Control       step_3      17416      23526     0.740287
  Control      confirm      15428      23526     0.655785
     Test        start      26672      26961     0.989281
     Test       step_1      24260      26961     0.899818
     Test       step_2      22252      26961     0.825340
     Test       step_3      20876      26961     0.774304
     Test      c